In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion

In [4]:
from sklearn.base import TransformerMixin,BaseEstimator

In [7]:
from sklearn.decomposition import pca

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

In [26]:
from sklearn.model_selection import train_test_split

In [9]:
from sklearn import svm
import sklearn.ensemble as ens

In [10]:
import os

In [11]:
os.chdir('dineral')

In [12]:
from internaldata import Database, Classifier

/home/tobi/numerics/envs/test_dineral3/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [13]:
clf = Classifier()

In [14]:
os.chdir('..')

In [15]:
db=Database()
data = db.load_data()

In [16]:
data.head()

,Hash,Datum,Text,Lastschrift,Deleted,Kategorie
0,3c3550b40b42a3367a2cb4b29d28412d,2014-01-03,01-37897-1 100000013241880130751593563\nAssur...,176.75,False,Krankenkasse
1,868f4d0ba29c129f408f979f3a9f8e0e,2014-01-03,BARGELDBEZUG\nVOM 01.01.2014\nKARTEN NR. 64186...,200.00,True,NaN
2,6848fb24b03ed71e56d98fd8f3af3401,2014-01-03,Zürcher Kantonalbank CH2300700350040352767 K...,35.00,False,Anschaffungen
3,5c42bb51a03c0c035f074a9a9b8ddcbb,2014-01-03,KAUF/DIENSTLEISTUNG\nVOM 31.12.2013\nKARTEN NR...,58.00,False,Ausgang
4,cf97222b176cb612026a061b0757d230,2014-01-05,ÜBERTRAG\nAUS KONTO 92-900275-2\nSCHOCH TOBIAS...,-1000.00,False,Sparen


In [17]:
class Scaler(BaseEstimator,TransformerMixin):
    
    def fit(self, X, y=None, **fit_params):
        self._factor = np.max(np.abs(X))
        return self
    
    def transform(self, X, y=None, **fit_params):
        return np.matrix((X/self._factor)).T

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

In [18]:
class ItemSelector(BaseEstimator,TransformerMixin):
    
    def __init__(self, key=0):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data):
        return data[self.key]

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

In [19]:
class DenseTransformer(BaseEstimator,TransformerMixin):

        def transform(self, X, y=None, **fit_params):
            return X.todense()

        def fit_transform(self, X, y=None, **fit_params):
            self.fit(X, y, **fit_params)
            return self.transform(X)

        def fit(self, X, y=None, **fit_params):
            return self

## Feature Pipeline

In [43]:
features = FeatureUnion([('text',Pipeline([('select',ItemSelector('Text')),
                                           ('vect',CountVectorizer(analyzer='char_wb',lowercase=True, strip_accents='unicode')),
                                           ('trans',TfidfTransformer(use_idf=True)),
                                           ('dense',DenseTransformer())])),
                         ('number',Pipeline([('select',ItemSelector('Lastschrift')),('scale',Scaler())]))])

In [46]:
features = FeatureUnion([('text',Pipeline([('select',ItemSelector('Text')),
                                           ('vect',CountVectorizer(analyzer='char_wb',lowercase=True, strip_accents='unicode')),
                                           ('trans',TfidfTransformer(use_idf=True)),
                                           ('dense',DenseTransformer())]))])

In [47]:
features = Pipeline([('vect',CountVectorizer(ngram_range=(4,5),analyzer='char_wb',lowercase=True, strip_accents='unicode')),
                     ('trans',TfidfTransformer(use_idf=True))])

### prepare data

In [22]:
data['Kategorie'] = data.Kategorie.cat.add_categories([u'Delete'])

In [23]:
data.Kategorie[data.Kategorie.isnull()]=u'Delete'

In [24]:
data.Kategorie[data.Deleted]=np.nan

In [25]:
categories = data.Kategorie.cat.categories
target = data.Kategorie.cat.codes

### test train split

In [28]:
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.05)

extract features

In [44]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    xtrain = features.fit_transform(data_train)
    xtest = features.fit_transform(data_test)

In [40]:
text_clf = Pipeline([('features',features),('clf',SGDClassifier)])

In [42]:
text_clf.fit(data_train, target_train, alpha=0.003)

ValueError: not enough values to unpack (expected 2, got 1)

In [36]:
text_clf.fit(data_train)

/home/tobi/numerics/envs/test_dineral3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


TypeError: fit() missing 1 required positional argument: 'y'

In [55]:
parameters = {
              'features__vect__ngram_range':[(i,j) for j in range(3,7) for i in range(1,j)],
              'clf__alpha': np.linspace(3e-3,1e-4,20)
             }

In [56]:
text_clf.fit(data_train['Text'],target_train)

/home/tobi/numerics/envs/test_dineral3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', Pipeline(steps=[('vect', CountVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(4, 5), preprocessor=None...     penalty='l2', power_t=0.5, random_state=12, shuffle=True, verbose=0,
       warm_start=False))])

## Grid search

In [57]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    gs_clf = GridSearchCV(text_clf,parameters,cv=5,verbose=True, n_jobs=8)
    gs_clf.fit(data_train['Text'],target_train)

Fitting 5 folds for each of 280 candidates, totalling 1400 fits


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    8.8s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   46.5s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:  3.3min
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:  5.1min
[Parallel(n_jobs=8)]: Done 1400 out of 1400 | elapsed:  5.8min finished


In [58]:
print(gs_clf.best_params_)

{'clf__alpha': 0.000710526315789474, 'features__vect__ngram_range': (1, 5)}


In [59]:
preds = gs_clf.predict(data_test['Text'])

/home/tobi/numerics/envs/test_dineral3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [38]:
preds_all = gs_clf.predict(data['Text'])

/home/tobi/numerics/envs/test_dineral3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [60]:
print(metrics.classification_report(categories[target_test],categories[preds]))

                 precision    recall  f1-score   support

  Anschaffungen       0.00      0.00      0.00         2
     Ausbildung       1.00      1.00      1.00         1
        Ausgang       0.78      0.88      0.82        16
     Bekleidung       0.00      0.00      0.00         1
         Bücher       0.50      1.00      0.67         1
         Delete       1.00      0.92      0.96        12
      Eishockey       1.00      0.80      0.89         5
          Essen       0.80      0.88      0.84        32
      Geschenke       1.00      0.33      0.50         3
       Hochzeit       0.00      0.00      0.00         1
   Krankenkasse       1.00      1.00      1.00         3
           Lohn       1.00      1.00      1.00         1
        Medizin       0.67      0.67      0.67         3
          Miete       1.00      1.00      1.00         6
   Mobiltelefon       1.00      1.00      1.00         3
Reisen/Ausflüge       0.85      1.00      0.92        11
       Schulden       1.00    

/home/tobi/numerics/envs/test_dineral3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [63]:
pd.DataFrame(metrics.confusion_matrix(categories[target_test],categories[preds],labels=categories),columns=categories,index=categories)

,Anschaffungen,Ausbildung,Ausgang,Bekleidung,Bussen,Bücher,Eishockey,Essen,Gebühren,Geschenke,...,Reisen/Ausflüge,Schulden,Sparen,Spenden,Sport allgemein,Steuern,Transport,Vorsorge,Wellness/Coiffeur,Delete
Anschaffungen,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
Ausbildung,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ausgang,0,0,14,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
Bekleidung,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Bussen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bücher,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Eishockey,0,0,0,0,0,0,4,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Essen,0,0,3,0,0,0,0,28,0,0,...,0,0,0,0,0,0,0,0,0,0
Gebühren,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Geschenke,0,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [63]:
import datetime

In [64]:
text_clf = gs_clf.best_estimator_
text_clf.classes_names = categories

text_clf.TRAINING_DATE = datetime.datetime.now()
text_clf.TRAINING_SAMPLES = data_train.shape[0]
text_clf.TEST_SAMPLES = data_test.shape[0]
text_clf.SCORE = metrics.precision_recall_fscore_support(target_test,preds)

In [65]:
import pickle
with open(clf.properties,"wb+") as fp:
    pickle.dump(text_clf,fp)